<a href="https://colab.research.google.com/github/beatricekiplagat/Deepfake-Audio-Recognition/blob/dev_branch/DEEPFAKE_AUDIO_DETECTION_TRANSFER_LEARNING_MODELS_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Preperation

In [1]:
!pip install wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import wandb
from wandb.keras import WandbCallback
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/DEEPFAKE AUDIO DETECTION PROJECT/DATASETS/Train')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('/content/drive/MyDrive/DEEPFAKE AUDIO DETECTION PROJECT/DATASETS/Train' + '/' +item)
 #print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
    #print(class_labels[:5])
    
    
# Build a dataframe        
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

['Spoof', 'Bonafide']
Types of classes labels found:  2
  Labels                                image
0  Spoof  dataset_path/Spoof/LA_D_7121651.png
1  Spoof  dataset_path/Spoof/LA_D_7212590.png
2  Spoof  dataset_path/Spoof/LA_D_6131320.png
3  Spoof  dataset_path/Spoof/LA_D_6692719.png
4  Spoof  dataset_path/Spoof/LA_D_8380959.png
        Labels                                           image
1645  Bonafide  dataset_path/Bonafide/Copy of LA_D_5945817.png
1646  Bonafide  dataset_path/Bonafide/Copy of LA_D_1696066.png
1647  Bonafide  dataset_path/Bonafide/Copy of LA_D_5226379.png
1648  Bonafide  dataset_path/Bonafide/Copy of LA_D_3440131.png
1649  Bonafide  dataset_path/Bonafide/Copy of LA_D_8126437.png


In [4]:
# Initialize wandb
wandb.login()

wandb: Currently logged in as: ruoro (use `wandb login --relogin` to force relogin)


True

In [5]:
wandb.init(project='DeepFake Audio Detection', entity='ruoro', save_code=True, 
           config = {
               'learning_rate': 0.001,
               'epochs': 7,
               'batch_size' : 4,
               'loss_function' : 'binary_crossentropy'
           })

In [6]:

# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  1650
Bonafide    851
Spoof       799
Name: Labels, dtype: int64


In [7]:
import cv2
path = '/content/drive/MyDrive/DEEPFAKE AUDIO DETECTION PROJECT/DATASETS/Train/'
dataset_path = os.listdir('/content/drive/MyDrive/DEEPFAKE AUDIO DETECTION PROJECT/DATASETS/Train')

im_size = 224

labels = [] #os.listdir(path)
images = [] #os.listdir(path + labels)


for i in dataset_path:
    data_path = path + str(i)  
    filenames = [i for i in os.listdir(data_path) ]
   
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [8]:
print(len(images))

1650


In [9]:
#This model takes input images of shape (224, 224, 3), and the input data should range [0, 255]. 

images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(1650, 224, 224, 3)

In [10]:
#Pre-processing the data. We need to perform label encoding in order to get the model to work
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['Spoof' 'Spoof' 'Spoof' ... 'Bonafide' 'Bonafide' 'Bonafide']
[1 1 1 ... 0 0 0]


In [11]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [12]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.15, random_state=42)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(1402, 224, 224, 3)
(1402, 2)
(248, 224, 224, 3)
(248, 2)


EfficientNet Implementation :

In [13]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 2
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


# Using model without transfer learning

outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

In [14]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/DEEPFAKE AUDIO DETECTION PROJECT/DATASETS/Test',
  target_size=(224, 224),
  batch_size=4,
  class_mode='categorical')

Found 400 images belonging to 2 classes.


In [ ]:
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

model.summary()

hist = model.fit(train_x, train_y, 
                 validation_data = (test_x,test_y),
                 epochs=30,
                 callbacks = [WandbCallback(data_type="image", labels=labels, generator=validation_generator)],
                 validation_steps= 20,
                 verbose=2)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 2)                 4052133   
Total params: 4,052,133
Trainable params: 4,010,110
Non-trainable params: 42,023
_________________________________________________________________
Epoch 1/30
44/44 - 47s - loss: 1.7705 - accuracy: 0.6605 - val_loss: 0.7264 - val_accuracy: 0.5484


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/30
44/44 - 25s - loss: 0.6614 - accuracy: 0.7910 - val_loss: 1.1102 - val_accuracy: 0.5484
Epoch 3/30


In [ ]:
import matplotlib.pyplot as plt

# loss
plt.plot(hist.history['loss'], label='train loss')
plt.plot(hist.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
# accuracies
plt.plot(hist.history['accuracy'], label='train accuracy')
plt.plot(hist.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

import tensorflow as tf

from tensorflow.keras.models import load_model


model.save('EffNetModel.pb')
# model.save('EffNetModel.h5')

In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
run = wandb.init(project='DeepFake Audio Detection', entity='ruoro')
model = wandb.Artifact('effnetmodel', type='model')
model.add_file('EffNetModel.h5')

run.log_artifact(model)

TESTING MODEL ON UNSEEN DATA :

In [ ]:
# from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input


# img_path = '/content/drive/MyDrive/DEEPFAKE AUDIO DETECTION PROJECT/DATASETS/Test/Spoof/LA_D_1115199.png'
img_path = '/content/drive/MyDrive/DEEPFAKE AUDIO DETECTION PROJECT/DATASETS/Test/Bonafide/LA_D_1364611.png'
#img = image.load_img(img_path, target_size=(224, 224))
#x = img.img_to_array(img)

img = cv2.imread(img_path)
img = cv2.resize(img, (224, 224))

x = np.expand_dims(img, axis=0)
x = preprocess_input(x)

print('Input image shape:', x.shape)

my_image = imread(img_path)
imshow(my_image)

In [ ]:
preds=model.predict(x)
preds     # probabilities for being in each of the 2 classes

In [ ]:
wandb.finish()